In [ ]:
import numpy as np
import tensorflow as tf
import pickle

data_path = 'model-data'

min_doc_length = 500
top_words = 10000

vocabdata_path = '%s/vocab3_books+reddit+wiki+stories+poems+iarpa.npz' % data_path
vocabdata = np.load(vocabdata_path)
words = set(vocabdata['svocab'][:top_words]) | set(vocabdata['storyvocab'])
vocab = map(str, list(words))
vocab.append('unk')
num_vocab = len(vocab)

int_to_word = {i: vocab[i] for i in xrange(num_vocab)}
word_to_int = {int_to_word[i]: i for i in xrange(num_vocab)}

# Un-comment if pickle doesn't already exist.
"""
from text.textcore.Corpus import Corpus
corpus_files = ['raw-transcripts/stories1.tar.gz', 'raw-transcripts/stories2.tar.gz', 
                'reddit/allsubs-20120423.txt.gz', 'wiki/wikipedia_txt.tar.gz']

corpus = Corpus('%s/'%data_path+corpus_files[2], min_length = min_doc_length, 
		   split_documents = 100000000, replace_uncommon=5)
# for i in xrange(3, 4):
#     corpus.append_corpus('%s/'%data_path+corpus_files[i])
documents = list(corpus.gen_document_arrays(vocab))
num_documents = len(documents)

with open('%s/reddit_docs_%d.pkl'%(data_path, min_doc_length), 'wb') as f:
    pickle.dump(documents, f, pickle.HIGHEST_PROTOCOL)
"""

with open('%s/reddit_docs_%d.pkl'%(data_path, min_doc_length), 'rb') as f:
    documents = pickle.load(f)

num_documents = len(documents)
print num_documents

for i in xrange(num_documents):
    documents[i] = np.array(documents[i], np.int32)
    documents[i][documents[i]==-1] = num_vocab - 1

In [ ]:
def cooccurrence(vocab, documents, window=15):
    num_vocab = len(vocab)
    word2int = {vocab[i] for i in xrange(num_vocab)}
    cooccur = np.zeros([num_vocab, num_vocab])
    for i in xrange(len(documents)):
        doc_int = documents[i]
#         doc_int = [word2int.get(word, 'unk') for word in doc]
        for w in xrange(window, len(doc_int)):
            w1 = doc_int[w]
            for w2 in doc_int[w-window:window]:
                cooccur[w1][w2] += 1
                cooccur[w2][w1] += 1
        if i % 10000 == 0: print i,
    return map(np.log, cooccur+1)

In [ ]:
C = cooccurrence(vocab, documents)

In [ ]:
safe_div = lambda a,b: a/b if b !=0 else np.zeros(a.shape[0])

#Z-score
E = map(safe_div, C - np.mean(C, axis=0), np.std(C, axis=0))
E = map(safe_div, E - np.mean(E, axis=1), np.std(E, axis=1))

E = np.array(E)

In [ ]:
print E